In [5]:
from utils.Weather_Data_by_API import *

In [1]:
import requests

class Data_by_API(object):
    
    def __init__(self, url):
        self.url = url
#         self.serviceKey = serviceKey
        
    def create_request_url(self, params_dict):
#         params_dict["service_key"] = self.serviceKey
        params_list = [f"{k}={v}" for k, v in params_dict.items()]
        params_str = "&".join(params_list)
#         print(params_str)
        
        self.request_url = self.url + params_str
        
        return self.request_url
    
    
    def request(self, request_url = None):
        
        if request_url == None:
            request_url = self.request_url
            
        rq = requests.get(request_url)
        
        return rq
        

In [16]:
import pandas as pd
import numpy as np
from collections import defaultdict
# from utils.Data_by_API import *



class Weather_Data_by_API(Data_by_API):
    
    base_url = "http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?"
    
    def __init__(self, params_dict):
        super().__init__(url = self.base_url)
        self.request_url = super().create_request_url(params_dict = params_dict)
        self.params_dict = params_dict
        
        
        
    def calculate_max_page(self):
        rq = super().request()
        
        self.n_rows = int(self.params_dict["numOfRows"])
        self.total_count = rq.json()["response"]["body"]["totalCount"]
        
        max_page = int(np.ceil(self.total_count / self.n_rows))
        
        print(f"n_rows : {self.n_rows}, total_count : {self.total_count}, max_page = {max_page}")
        
        return max_page
    
    def get(self):
        rq = super().request()
        
        max_page = self.calculate_max_page()
        
        output_cols = rq.json()["response"]["body"]["items"]["item"][0].keys()
        
        data_dict = defaultdict(list)
        for i in range(max_page):
            
            self.params_dict["pageNo"] = i + 1
            
            request_url = super().create_request_url(params_dict = self.params_dict)
            rq = self.request(request_url = request_url)
            json_list = rq.json()["response"]["body"]["items"]["item"]
                                                               
            for js in json_list:
                for col in output_cols:
                    data_dict[col].append(js[col])
                    
        return pd.DataFrame(data_dict)

In [17]:
service_key = "eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D"

In [18]:
params_dict = {"serviceKey" : service_key,
                "stnIds" : "152",
                "startDt" : "20200101",
                "startHh" : "00",
                "endDt" : "20201201", 
                "endHh" : "23",
                "numOfRows" : "900",
                "dataType" : "JSON",
                "pageNo" : "1",
                "dataCd" : "ASOS",
                "dateCd" : "HR"
                }

In [19]:
weather = Weather_Data_by_API(params_dict = params_dict)

In [20]:
%%time
weather_data = weather.get()

n_rows : 900, total_count : 8064, max_page = 9
CPU times: user 144 ms, sys: 1.79 ms, total: 146 ms
Wall time: 17.5 s


In [6]:
%%time
weather_data = weather.get()

n_rows : 900, total_count : 8064, max_page = 9
CPU times: user 141 ms, sys: 3.44 ms, total: 144 ms
Wall time: 18.5 s


In [7]:
weather_data.drop_duplicates()

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,2020-01-01 00:00,1,152,울산,-2.7,,,,2.3,,...,,2000,,,-4.0,,,,,
1,2020-01-01 01:00,2,152,울산,-3.3,,,,2.2,,...,,2000,,,-4.3,,,,,
2,2020-01-01 02:00,3,152,울산,-3.6,,,,3.0,,...,,2000,,,-4.4,,,,,
3,2020-01-01 03:00,4,152,울산,-3.3,,,,2.4,,...,,2000,,,-4.1,,,,,
4,2020-01-01 04:00,5,152,울산,-3.1,,,,1.3,,...,,2000,,,-4.0,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,2020-12-01 19:00,8060,152,울산,6.8,,,,2.2,,...,74,2000,,,4.0,,,,,
8060,2020-12-01 20:00,8061,152,울산,6.1,,,,2.2,,...,70,2000,,,3.2,,,,,
8061,2020-12-01 21:00,8062,152,울산,5.8,,,,1.9,,...,,1998,,,2.4,,,,,
8062,2020-12-01 22:00,8063,152,울산,4.5,,,,1.7,,...,,2000,,,1.8,,,,,


In [8]:
weather_data.to_parquet("~/data/weather_2018.parquet")